In [1]:
import duckdb
import leafmap
import pandas as pd
con = duckdb.connect()
con.install_extension("httpfs")
con.load_extension("httpfs")
con.install_extension("spatial")
con.load_extension("spatial")

In [9]:
sql = ("""
    CREATE TABLE lewo AS
    SELECT *,
        ST_Point(LONGITUDE, LATITUDE) as geometry
    FROM "./test/lewo.csv")
""")
con.execute(sql)


ParserException: Parser Error: syntax error at or near ")"
LINE 5:     FROM "./test/lewo.csv")
                                  ^

In [61]:
con.table("lewo")

┌──────────┬──────────────────┬──────────────────────┬───┬───────────────┬──────────┬──────────────────────┐
│ OBJECTID │      STATE       │        COUNTY        │ … │ LOCALITY_TYPE │ geometry │      geometry:1      │
│  int64   │     varchar      │       varchar        │   │    varchar    │ varchar  │       geometry       │
├──────────┼──────────────────┼──────────────────────┼───┼───────────────┼──────────┼──────────────────────┤
│        1 │ Oregon           │ Jackson              │ … │ P             │ NULL     │ POINT (-122.807327…  │
│        2 │ Washington       │ Klickitat            │ … │ P             │ NULL     │ POINT (-120.503788…  │
│        3 │ California       │ Yolo                 │ … │ P             │ NULL     │ POINT (-122.043857…  │
│        4 │ California       │ Tehama               │ … │ H             │ NULL     │ POINT (-122.367568…  │
│        5 │ Colorado         │ Costilla             │ … │ P             │ NULL     │ POINT (-105.536073…  │
│        6 │ Califo

In [62]:
con.sql('''
    SELECT STATE, COUNT(OBJECTID) as Count
    FROM lewo
    GROUP BY STATE
    ORDER BY Count DESC
    LIMIT 10
''')

┌──────────────────┬───────┐
│      STATE       │ Count │
│     varchar      │ int64 │
├──────────────────┼───────┤
│ California       │ 42674 │
│ Colorado         │ 18383 │
│ Oregon           │ 17694 │
│ Washington       │ 12389 │
│ Arizona          │ 11299 │
│ British Columbia │  8675 │
│ Montana          │  7682 │
│ New Mexico       │  7092 │
│ Idaho            │  6058 │
│ Utah             │  3018 │
├──────────────────┴───────┤
│ 10 rows        2 columns │
└──────────────────────────┘

In [63]:
# Select only the Washington state points from the table and call the new table wash
con.sql('''
    CREATE TABLE wash AS
    SELECT *
    FROM lewo
    WHERE STATE = 'Washington'
''')

In [64]:
# Export the wash table to a csv file
# Fetch the data from the wash table into a DataFrame
wash_df = con.execute("SELECT * FROM wash").fetchdf()

# Export the DataFrame to a CSV file in the test folder
wash_df.to_csv('test/wash.csv', index=False)

In [65]:
print(wash_df)

       OBJECTID       STATE     COUNTY   LATITUDE   LONGITUDE COUNT   DATE  \
0             2  Washington  Klickitat  45.893689 -120.503788     2  35609   
1            12  Washington     Benton  46.502646 -119.712353     1  35196   
2            23  Washington     Yakima  46.850000 -120.800000     X  32277   
3            69  Washington  Klickitat  45.744910 -120.436986     2  35511   
4           108  Washington      Ferry  47.942785 -118.690700     2  34148   
...         ...         ...        ...        ...         ...   ...    ...   
12384    140020  Washington   Okanogan  48.845603 -119.662783     1  45094   
12385    140046  Washington   Skamania  45.818325 -121.758092     2  45059   
12386    140053  Washington     Yakima  46.904527 -120.724860     3  45056   
12387    140060  Washington   Kittitas  46.918207 -120.732669     1  45073   
12388    140073  Washington     Yakima  46.343056 -120.839444     3  45066   

                                              LOCALITY LOCALITY